<a href="https://colab.research.google.com/github/Uttpal-Tripathy/DDoS-Attack-detection/blob/main/DDPG_based_DDoS_attack_detection_in_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/cicddos2019_dataset.csv')
data

,Unnamed: 0,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Fwd Packets Length Total,Bwd Packets Length Total,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Class
0,0,17,216631,6,0,2088.0,0.0,393.0,321.0,348.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
1,1,17,2,2,0,802.0,0.0,401.0,401.0,401.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
2,2,17,48,2,0,766.0,0.0,383.0,383.0,383.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
3,3,17,107319,4,0,1398.0,0.0,369.0,330.0,349.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
4,4,17,107271,4,0,1438.0,0.0,389.0,330.0,359.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5892,5892,17,109195,4,0,1438.0,0.0,389.0,330.0,359.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
5893,5893,17,107539,4,0,1398.0,0.0,369.0,330.0,349.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
5894,5894,17,105380,4,0,1398.0,0.0,369.0,330.0,349.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack
5895,5895,17,55,4,0,1802.0,0.0,472.0,429.0,450.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UDP,Attack


In [ ]:
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['Label'])

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# 1. Drop the 'Label' column *before* selecting categorical features
X = data.drop('Label', axis=1)  # Assuming 'Label' is your target variable
y = data['Label']

# 2. Select categorical features from the modified X
categorical_features = X.select_dtypes(include=['object']).columns

# 3. Create a ColumnTransformer to handle numeric and categorical features separately
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(exclude=['object']).columns), # Apply StandardScaler to numeric features
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features) # Apply OneHotEncoder to categorical features
    ])

# 4. Fit and transform the data
X_scaler = preprocessor.fit_transform(X)

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=0)


In [ ]:
print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (4717, 80)
Test set size: (1180, 80)


In [ ]:
import gym
from gym import spaces
class DDoSEnv(gym.Env):
    def __init__(self, X_train, y_train, X_test, y_test):
        super(DDoSEnv, self).__init__()
        self.X_train = X_train
        self.y_train = y_train.values  # Convert y_train to numpy array
        self.X_test = X_test
        self.y_test = y_test.values  # Convert y_test to numpy array
        self.current_step = 0
        self.action_space = spaces.Box(
            low=-1, high=1, shape=(1,), dtype=np.float32
        )  # Example action space
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(X_train.shape[1],),
            dtype=np.float32,
        )  # Example observation space

    def reset(self):
        self.current_step = 0
        return self.X_train[self.current_step]

    def step(self, action):
        # Implement your reward function here based on the action and the environment state
        # Example reward function
        reward = -np.abs(action - self.y_train[self.current_step])
        self.current_step += 1
        done = self.current_step >= len(self.X_train)
        info = {}
        obs = (
            self.X_train[self.current_step]
            if not done
            else self.X_test[0]  # Return next observation or reset
        )
        return obs, reward, done, info

In [ ]:
# prompt: Build DDPG model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DDPGAgent:
    def __init__(self, state_dim, action_dim, action_bound):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.action_bound = action_bound

        self.actor = self.build_actor()
        self.critic = self.build_critic()

    def build_actor(self):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_dim=self.state_dim))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_dim, activation='tanh'))  # Output layer with tanh activation
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    def build_critic(self):
        model = Sequential()
        model.add(Dense(64, activation='relu', input_dim=self.state_dim + self.action_dim))  # Input includes both state and action
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1, activation='linear'))  # Output a single value (Q-value)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    def get_action(self, state):
        action = self.actor.predict(state)
        # Scale the action to the environment's action bounds
        return action * self.action_bound

# Example usage (assuming you have state_dim and action_bound from your environment)
state_dim = X_train.shape[1]  # Replace with actual state dimension
action_dim = 1  # Example: 1 continuous action
action_bound = 1.0  # Replace with actual action bound

agent = DDPGAgent(state_dim, action_dim, action_bound)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
env = DDoSEnv(X_train, y_train, X_test, y_test)

In [ ]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
!pip install shimmy>=2.0

In [ ]:
import numpy as np
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise

class DDoSEnv(gym.Env):
    def __init__(self, X_train, y_train, X_test, y_test):
        super(DDoSEnv, self).__init__()
        self.X_train = X_train
        self.y_train = y_train.values # Convert y_train to numpy array
        self.X_test = X_test
        self.y_test = y_test.values # Convert y_test to numpy array
        self.current_step = 0
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32) # Example action space
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(X_train.shape[1],), dtype=np.float32) # Example observation space

    def reset(self):
        self.current_step = 0
        return self.X_train[self.current_step]

    def step(self, action):
        # Implement your reward function here based on the action and the environment state
        # Example reward function
        reward = -np.abs(action - self.y_train[self.current_step])
        self.current_step += 1
        done = self.current_step >= len(self.X_train)
        info = {}
        obs = self.X_train[self.current_step] if not done else self.X_test[0]  # Return next observation or reset
        return obs, reward, done, info

# Assuming you have your environment set up and other necessary components as shown in the provided code
env = DDoSEnv(X_train, y_train, X_test, y_test)

# The noise objects for DDPG
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# Create the DDPG model
model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)

# Train the agent
model.learn(total_timesteps=10000) # Adjust timesteps as needed

# Save the model
model.save("ddpg_ddos_model")

# Load the model
# model = DDPG.load("ddpg_ddos_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/monitor.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.rewards.append(float(reward))
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/dummy_vec_env.py:59: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.

In [ ]:

from stable_baselines3.common.evaluation import evaluate_policy


mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10) # Adjust n_eval_episodes as needed

print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -40.00 +/- 0.00


In [ ]:


from sklearn.metrics import confusion_matrix, classification_report


y_pred = model.predict(X_test)[0]


y_pred_binary = [1 if val >= 0 else 0 for val in y_pred]

#
print(classification_report(y_test, y_pred_binary))


conf_matrix = confusion_matrix(y_test, y_pred_binary)


if conf_matrix.shape == (2, 2):
    tn, fp, fn, tp = conf_matrix.ravel()
    print(f"True Negatives (TN): {tn}")
    print(f"True Positives (TP): {tp}")
    print(f"False Negatives (FN): {fn}")
    print(f"False Positives (FP): {fp}")
else:
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Note: This is not a binary classification problem. TN, FP, FN, TP cannot be directly extracted.")

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.99      1.00      1.00      1168
           2       0.00      0.00      0.00         1

    accuracy                           0.99      1180
   macro avg       0.33      0.33      0.33      1180
weighted avg       0.98      0.99      0.99      1180

Confusion Matrix:
[[   0   11    0]
 [   0 1168    0]
 [   1    0    0]]
Note: This is not a binary classification problem. TN, FP, FN, TP cannot be directly extracted.


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
